In [ ]:
import requests
import yfinance as yf
import pandas as pd
import xlwings as xw
import time



#創新活頁簿
new_wb = xw.Book()

#定義算kd值
def calculate_kd(data, n=8):
    rsv = (data['Close'] - data['Low'].rolling(window=n).min()) / (data['High'].rolling(window=n).max() - data['Low'].rolling(window=n).min()) * 100
    k = pd.Series(0.0, index=data.index)
    d = pd.Series(0.0, index=data.index)
    for i in range(n, len(data)):
        k[i] = (2/3) * k[i-1] + (1/3) * rsv[i]
        d[i] = (2/3) * d[i-1] + (1/3) * k[i]

    data['K'], data['D'] = k, d
    return data
#定義算第一天值
def calculate_first_day(sheet):
    price = sheet.range("B10").value
    k = sheet.range("E10").value
    d = sheet.range("F10").value
    shares = sheet.range("M2").value = 10
    capital = sheet.range("L2").value = 10000
    sheet.range("G10").value = shares if k < 40 and k > d else 0
    sheet.range("H10").value = shares if k > 60 and k < d else 0
    sheet.range("I10").value = sheet.range("G10").value - sheet.range("H10").value
    sheet.range("J10").value = capital - sheet.range("I10").value * sheet.range("B10").value
    sheet.range("K10").value = sheet.range("J10").value + sheet.range("I10").value * sheet.range("B10").value
#定義回測
def perform_financial_calculations(sheet):
    last_row = sheet.range("A1").end("down").row
    shares = sheet.range("M2").value
    capital = sheet.range("L2").value = 10000
    for i in range(11, last_row + 1):
        sheet.range(f"G{i}").formula = f"=IF(AND(E{i}<=40, E{i}>=F{i}, B{i}*M2<=J{i-1}), M2, 0)"
        sheet.range(f"H{i}").formula = f"=IF(AND(E{i}>=60, E{i}<=F{i}, I{i-1}>=M2), M2, 0)"
        sheet.range(f"I{i}").formula = f"=G{i}-H{i}+I{i-1}"
        sheet.range(f"J{i}").formula = f"=(H{i}-G{i})*B{i}+J{i-1}"
        sheet.range(f"K{i}").formula = f"=I{i}*B{i}+J{i}"
        sheet.range(f"N2").formula = f"=K{i}-L2"

#抓歷史資料及算kd值、第一天值、回測並匯入

#抓S&P500值前15名
url = 'https://www.slickcharts.com/sp500'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
stock_symbols = pd.read_html(requests.get(url, headers=headers).text)[0]['Symbol'].str.replace('.', '-').head(15)

#歷史資料迴圈
for symbol in stock_symbols:
    try:
        history = yf.Ticker(symbol).history(period='12mo')
        headers = ["Date", "Close", "High", "Low", "K", "D", "Buy", "Sell", "Hold", "Cash", "Assets", "Capital", "Share", "TotalRevenue"]

        #算kd
        history = calculate_kd(history)

        #匯入表頭及歷史資料
        new_sheet = new_wb.sheets.add(name=symbol)
        new_sheet.range("A1").value = headers
        new_sheet.range("A1").value = history[['Close', 'High', 'Low', 'K', 'D']]

        #回測及算第一天
        perform_financial_calculations(new_sheet)
        calculate_first_day(new_sheet)

        print(f"Processed data for {symbol} successfully.")

        #避免短暫多次request
        time.sleep(1)
    #如果列印失敗，印出traceback以利debug
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"Error processing data for {symbol}: {e}")

#刪除工作表1以免後續portfolio無法進行
new_wb.sheets['工作表1'].delete()
#存新工作簿
new_wb.save(r"399_CTW.xlsx")


# 收益
profit = 0
# 成本
cost = 0
# 起始 row 值
row = 2
portfolio = new_wb.sheets.add(name="portfolio")
portfolio.range("A1").value = "StockName"
portfolio.range("B1").value = "StockProfit（$）"
portfolio.range("C1").value = "StockProfit（%）"
portfolio.range("A17").value = "TotalProfit（$）"
portfolio.range("C17").value = "TotalProfit（%）"

for sheet in new_wb.sheets:
    # 判斷目前迭代到的工作表是否為portfolio
    if sheet.name == "portfolio":
        # 若是，就不執行回測
        continue
    # 將目前StockName寫入portfolio A 欄
    portfolio.range(f"A{row}").value = sheet.name
    # 將目前工作表的StcokProfit寫入portfolio B 欄
    portfolio.range(f"B{row}").value = sheet.range("N2").value

    #改為趴數
    #cell_range = wb.sheets[0].range('A1:A10')
    #cell_range.number_format = '0%'

    # 將目前工作表的收益比例寫入portfolio C 欄
    portfolio.range(f"C{row}").value = sheet.range("N2").value / sheet.range("L2").value
    # 更新 row 值
    row += 1
    # 叠加總成本金額
    cost += sheet.range("L2").value
    # 叠加總收益金額
    profit += sheet.range("N2").value
#更新TotalProfit($)值
portfolio.range("B17").value = profit
#更新TotalProfit(%)值
portfolio.range("D17").value = profit / cost

Processed data for AAPL successfully.
